## Увод

Ова лекција ће обухватити:
- Шта је позивање функције и њене примене
- Како направити позив функције користећи OpenAI
- Како интегрисати позив функције у апликацију

## Циљеви учења

Након завршетка ове лекције знаћете како и разумети:

- Сврху коришћења позивања функција
- Подешавање позива функције користећи OpenAI сервис
- Дизајнирање ефикасних позива функција за примену у вашој апликацији


## Разумевање позива функција

За ову лекцију, желимо да направимо функцију за наш едукативни стартап која омогућава корисницима да користе ћаскање са ботом како би пронашли техничке курсеве. Препоручићемо курсеве који одговарају њиховом нивоу вештина, тренутној улози и технологији од интереса.

Да бисмо то завршили, користићемо комбинацију:
 - `OpenAI` за креирање искуства ћаскања за корисника
 - `Microsoft Learn Catalog API` да помогнемо корисницима да пронађу курсеве на основу захтева корисника
 - `Function Calling` да узмемо упит корисника и пошаљемо га функцији која ће направити API захтев.

Да бисмо почели, погледајмо зашто бисмо уопште желели да користимо позив функције:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # добијање новог одговора од GPT где може видети одговор функције


print(second_response.choices[0].message)


### Зашто позив функције

Ако сте завршили било коју другу лекцију у овом курсу, вероватно разумете моћ коришћења великих језичких модела (LLMs). Надамо се да такође можете видети и неке од њихових ограничења.

Позив функције је функција OpenAI сервиса дизајнирана да реши следеће изазове:

Непоуздано форматирање одговора:
- Пре позива функције, одговори великог језичког модела били су неструктурирани и непоуздани. Програмери су морали да пишу сложен код за валидацију како би обрадили сваку варијацију у излазу.

Ограничена интеграција са спољним подацима:
- Пре ове функције, било је тешко укључити податке из других делова апликације у контекст ћаскања.

Стандартизовањем формата одговора и омогућавањем беспрекорне интеграције са спољним подацима, позив функције поједностављује развој и смањује потребу за додатном логиком валидације.

Корисници нису могли добити одговоре као што је „Какво је тренутно време у Стокхолму?“. То је зато што су модели били ограничени на време када су подаци били обучавани.

Погледајмо пример испод који илуструје овај проблем:

Рецимо да желимо да креирамо базу података о студентима како бисмо им могли предложити прави курс. Испод имамо два описа студената која су веома слична у подацима које садрже.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Желимо да ово пошаљемо LLM-у да би парсирао податке. Ово касније може да се користи у нашој апликацији за слање овога на API или чување у бази података.

Хајде да направимо два идентична упита у којима ћемо LLM-у дати инструкције о томе које информације нас занимају:


Желимо да пошаљемо ово LLM-у да анализира делове који су важни за наш производ. Тако да можемо направити два идентична упита да упутимо LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Након креирања ова два упита, послаћемо их LLM-у користећи `openai.ChatCompletion`. Садржај упита чувамо у променљивој `messages` и додељујемо улогу `user`. Ово је да би се имитирало слање поруке од стране корисника ка ћаскању са ботом.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Сада можемо послати оба захтева LLM-у и испитати одговор који добијемо.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Иако су упити исти и описи слични, можемо добити различите формате својства `Grades`.

Ако покренете горњу ћелију више пута, формат може бити `3.7` или `3.7 GPA`.

То је зато што LLM узима неструктуриране податке у облику написаног упита и враћа такође неструктуриране податке. Потребно је да имамо структуриран формат како бисмо знали шта да очекујемо приликом чувања или коришћења тих података.

Коришћењем функционалног позивања, можемо бити сигурни да ћемо добити структуриран податак назад. Када користимо функционално позивање, LLM заправо не позива или извршава било какве функције. Уместо тога, креирамо структуру коју LLM треба да прати у својим одговорима. Затим користимо те структуриранe одговоре да бисмо знали коју функцију да покренемо у нашим апликацијама.


![Дијаграм тока позива функције](../../../../translated_images/sr/Function-Flow.083875364af4f4bb.png)


Затим можемо узети оно што функција врати и послати то назад LLM-у. LLM ће затим одговорити користећи природни језик како би одговорио на кориснички упит.


### Примери употребе позива функција

**Позивање спољних алата**  
Чатботови су одлични у пружању одговора на питања корисника. Коришћењем позива функција, чатботови могу користити поруке корисника да обаве одређене задатке. На пример, студент може затражити од чатбота да „Пошаље имејл мом предавачу у коме каже да ми је потребна додатна помоћ око овог предмета“. Ово може изазвати позив функције `send_email(to: string, body: string)`

**Креирање API или упита базе података**  
Корисници могу пронаћи информације користећи природни језик који се претвара у форматирани упит или API захтев. Пример за то може бити наставник који тражи „Који су студенти завршили последњи задатак“ што може изазвати позив функције `get_completed(student_name: string, assignment: int, current_status: string)`

**Креирање структурираних података**  
Корисници могу узети блок текста или CSV и користити LLM да из њега извуче важне информације. На пример, студент може претворити Википедијин чланак о мировним споразумима у AI флаш картице. Ово се може урадити коришћењем функције `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Креирање Вашег Првог Позива Функције

Процес креирања позива функције укључује 3 главна корака:
1. Позивање Chat Completions API-ја са списком ваших функција и поруком корисника
2. Читање одговора модела да бисте извршили акцију, нпр. извршили функцију или API позив
3. Још један позив Chat Completions API-ју са одговором из ваше функције да бисте користили те информације за креирање одговора кориснику.


![Ток позива функције](../../../../translated_images/sr/LLM-Flow.3285ed8caf4796d7.png)


### Елементи позива функције

#### Кориснички унос

Први корак је креирање корисничке поруке. Ово се може динамички доделити узимањем вредности из текстуалног уноса или можете овде доделити вредност. Ако је ово први пут да радите са Chat Completions API-јем, потребно је да дефинишемо `role` и `content` поруке.

`role` може бити или `system` (креирање правила), `assistant` (модел) или `user` (крајњи корисник). За позив функције, доделићемо ово као `user` и пример питања.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Креирање функција.

Следеће ћемо дефинисати функцију и параметре те функције. Овде ћемо користити само једну функцију која се зове `search_courses`, али можете креирати више функција.

**Важно** : Функције су укључене у системску поруку за LLM и урачунате су у укупан број доступних токена које имате на располагању.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Дефиниције** 

Структура дефиниције функције има више нивоа, сваки са својим својствима. Ево прегледа угнежђене структуре:

**Својства функције на највишем нивоу:**

`name` - Име функције коју желимо да позовемо.

`description` - Ово је опис како функција ради. Важно је да буде прецизан и јасан.

`parameters` - Листа вредности и формата које желите да модел произведе у свом одговору.

**Својства објекта параметара:**

`type` - Тип података објекта параметара (обично "object").

`properties` - Листа специфичних вредности које ће модел користити за свој одговор.

**Својства појединачних параметара:**

`name` - Имплицитно дефинисано кључем својства (нпр. "role", "product", "level").

`type` - Тип података овог конкретног параметра (нпр. "string", "number", "boolean").

`description` - Опис конкретног параметра.

**Опциона својства:**

`required` - Низ који наводи који су параметри неопходни да би позив функције био извршен.


### Позивање функције  
Након дефинисања функције, сада је потребно укључити је у позив Chat Completion API-ја. То радимо додавањем `functions` у захтев. У овом случају `functions=functions`. 

Постоји и опција да се `function_call` подеси на `auto`. То значи да ћемо дозволити LLM-у да одлучи која функција треба да буде позвана на основу корисничке поруке, уместо да то ми сами одредимо.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Хајде сада да погледамо одговор и видимо како је форматиран:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Можете видети да је име функције позвано и да је LLM из поруке корисника успео да пронађе податке који одговарају аргументима функције.


## 3.Интеграција позива функција у апликацију. 


Након што смо тестирали форматирани одговор од LLM-а, сада можемо то интегрисати у апликацију. 

### Управљање током 

Да бисмо то интегрисали у нашу апликацију, хајде да предузмемо следеће кораке: 

Прво, хајде да позовемо OpenAI сервисе и сачувамо поруку у променљиву која се зове `response_message`. 


In [ ]:
response_message = response.choices[0].message

Сада ћемо дефинисати функцију која ће позивати Microsoft Learn API да добије листу курсева:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Као најбоља пракса, затим ћемо видети да ли модел жели да позове функцију. Након тога, креираћемо једну од доступних функција и упоредити је са функцијом која се позива.  
Затим ћемо узети аргументе функције и мапирати их на аргументе из LLM-а.

На крају, додаћемо поруку о позиву функције и вредности које су враћене поруком `search_courses`. Ово даје LLM-у све информације које су му потребне да одговори кориснику користећи природни језик.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Сада ћемо послати ажурирану поруку LLM-у како бисмо добили одговор у природном језику уместо одговора форматираног као API JSON.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Изазов са кодом

Одличан посао! Да бисте наставили са учењем о OpenAI Function Calling, можете направити: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Више параметара функције који могу помоћи ученицима да пронађу више курсева. Доступне API параметре можете пронаћи овде:  
 - Направите још један позив функције који узима више информација од ученика, као што је њихов матерњи језик  
 - Направите обраду грешака када позив функције и/или API позив не врати одговарајуће курсеве


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Одрицање од одговорности**:
Овај документ је преведен коришћењем AI услуге за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да превод буде тачан, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитетним извором. За критичне информације препоручује се професионални људски превод. Нисмо одговорни за било каква неспоразума или погрешна тумачења која произилазе из коришћења овог превода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
